### Introduction:

In this code, we will find some nash equilibriums, under the combinations of the following situations:

- Government Strategies:
    - general_tax_scenarios: No General Tax, 10% General Tax, 11.25% General Tax
    - low_pricing_allowance_scenarios_SAEV: SAEV low pricing allowance 0, SAEV low pricing allowance 2, SAEV low pricing allowance 4
    - low_pricing_allowance_scenarios_PT: PT low pricing allowance 0, PT low pricing allowance 1
    - infrastructure_allowance_scenarios: No Infrastructure Allowance, Infrastructure Allowance
    - fixed_allowance_scenarios: No Fixed Allowance, Fixed Allowance
    - variable_allowance_scenarios: No Variable Allowance, 33% Variable Allowance, 41% Variable Allowance

- Power Grid Strategies:
    - pricing_scenarios: Real Time, Spatial Temporal

- SAEV strategies:
    - rebalancing_options: Non-Rebalancing, Rebalancing
    - discharging_options: Non-Discharging, Discharging

And for each case, the nash equilibrium is estimated for the following pricing strategies:

- SAEV: {1.75, 2}, price for one kilometer.
- PT: {2.5, 3}, price for one ticket.

In this version, we set **general_tax_scenarios** to be 11.25% General Tax, **infrastructure_allowance_scenarios** to be No Infrastructure Allowance, **fixed_allowance_scenarios** to be No Fixed Allowance, and **variable_allowance_scenarios** to be No Variable Allowance. Also, we set **SAEV strategies** to be Rebalancing and Discharging. Then we iterate over the 12 combinations of 3 **low_pricing_allowance_scenarios_SAEV strategies**, 2 **low_pricing_allowance_scenarios_PT strategies** and 2 **Power Grid Strategies**, to see the impact of low pricing allowances on Nash equilibrium.

In [5]:
import numpy as np
import pandas as pd
import public_transport_payoff as ptp

# Define the data structure for SAEV payoff
SAEV_payoff = {
    "SAEV Pricing 1.75": {
        "PT Pricing 2.5": {
            "Real Time": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -11.28},
                    "Discharging": {"Income (Pre-Tax)": -10.14}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 26.82},
                    "Discharging": {"Income (Pre-Tax)": 26.83}
                }
            },
            "Spatial Temporal": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -10.87},
                    "Discharging": {"Income (Pre-Tax)": -7.19}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 27.86},
                    "Discharging": {"Income (Pre-Tax)": 28.57}
                }
            }
        },
        "PT Pricing 3": {
            "Real Time": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -11.18},
                    "Discharging": {"Income (Pre-Tax)": -10.07}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 27.03},
                    "Discharging": {"Income (Pre-Tax)": 27.04}
                }
            },
            "Spatial Temporal": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -10.78},
                    "Discharging": {"Income (Pre-Tax)": -7.12}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 28.07},
                    "Discharging": {"Income (Pre-Tax)": 28.77}
                }
            }
        }
    },
    "SAEV Pricing 2": {
        "PT Pricing 2.5": {
            "Real Time": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -5.84},
                    "Discharging": {"Income (Pre-Tax)": -4.80}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 28.54},
                    "Discharging": {"Income (Pre-Tax)": 28.54}
                }
            },
            "Spatial Temporal": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -5.46},
                    "Discharging": {"Income (Pre-Tax)": -2.05}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 29.47},
                    "Discharging": {"Income (Pre-Tax)": 30.09}
                }
            }
        },
        "PT Pricing 3": {
            "Real Time": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -6.58},
                    "Discharging": {"Income (Pre-Tax)": -5.50}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 30.27},
                    "Discharging": {"Income (Pre-Tax)": 30.27}
                }
            },
            "Spatial Temporal": {
                "Non-Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": -6.20},
                    "Discharging": {"Income (Pre-Tax)": -2.65}
                },
                "Rebalancing": {
                    "Non-Discharging": {"Income (Pre-Tax)": 31.26},
                    "Discharging": {"Income (Pre-Tax)": 31.93}
                }
            }
        }
    }
}

# Define the pricing strategy combinations
pt_demand_files = {
    '1.75,2.5': '1.75,2.5.xlsx',
    '1.75,3': '1.75,3.xlsx',
    '2,2.5': '2,2.5.xlsx',
    '2,3': '2,3.xlsx'
}

saev_demand_files = {
    '1.75,2.5': 'saev_1.75,2.5.xlsx',
    '1.75,3': 'saev_1.75,3.xlsx',
    '2,2.5': 'saev_2,2.5.xlsx',
    '2,3': 'saev_2,3.xlsx'
}

# general_tax_rate = 0.1125
high_pricing_tax_rate_SAEV = 0.1

fix_allowance_amount = 1

# Function to levy general tax on terms larger than zero and return the taxed matrix and total tax
def apply_general_tax(payoff_matrix, general_tax_rate):
    taxed_matrix = np.copy(payoff_matrix)
    tax_matrix = np.zeros_like(payoff_matrix)
    for i in range(taxed_matrix.shape[0]):
        for j in range(taxed_matrix.shape[1]):
            if taxed_matrix[i, j] > 0:
                tax = taxed_matrix[i, j] * general_tax_rate
                taxed_matrix[i, j] -= tax
                tax_matrix[i, j] = tax
    return taxed_matrix, tax_matrix

# Function to levy high pricing tax on player 1 when the pricing is at the large strategy
def apply_high_pricing_tax_U1(payoff_matrix):
    taxed_matrix = np.copy(payoff_matrix)
    tax_matrix = np.zeros_like(payoff_matrix)
    for j in range(taxed_matrix.shape[1]):
        tax = np.abs(taxed_matrix[taxed_matrix.shape[0] - 1, j] * high_pricing_tax_rate_SAEV)
        taxed_matrix[taxed_matrix.shape[0] - 1, j] -= tax
        tax_matrix[taxed_matrix.shape[0] - 1, j] = tax
    return taxed_matrix, tax_matrix

# Function to levy high pricing tax on player 2 when the pricing is at the large strategy
def apply_high_pricing_tax_U2(payoff_matrix, high_pricing_tax_rate_PT):
    taxed_matrix = np.copy(payoff_matrix)
    tax_matrix = np.zeros_like(payoff_matrix)
    for i in range(taxed_matrix.shape[0]):
        tax = np.abs(taxed_matrix[i, taxed_matrix.shape[1] - 1] * high_pricing_tax_rate_PT)
        taxed_matrix[i, taxed_matrix.shape[1] - 1] -= tax
        tax_matrix[i, taxed_matrix.shape[1] - 1] = tax
    return taxed_matrix, tax_matrix

# Calculate the payoffs for public transport under different scenarios
U2_full = np.zeros((2, 2))
infrastructure_cost_matrix = np.zeros((2, 2))
variable_cost_matrix = np.zeros((2, 2))
gov_env_matrix = np.zeros((2, 2))


for (i, (saev_pricing, pt_pricing_data)) in enumerate(SAEV_payoff.items()):
    for (j, (pt_pricing, electricity_pricing_data)) in enumerate(pt_pricing_data.items()):
        pt_demand_file = pt_demand_files[f'{saev_pricing.split()[-1]},{pt_pricing.split()[-1]}']
        saev_demand_file = saev_demand_files[f'{saev_pricing.split()[-1]},{pt_pricing.split()[-1]}']
        U2_full[i, j] = ptp.calculate_payoff(pt_demand_file)[0] / 1e7
        infrastructure_cost_matrix[i, j] = ptp.calculate_payoff(pt_demand_file)[1] / 1e7
        variable_cost_matrix[i, j] = ptp.calculate_payoff(pt_demand_file)[2] / 1e7
        gov_env_matrix[i, j] = ptp.gov_env(saev_demand_file, pt_demand_file) / 1e7

# Function to find Nash equilibria
def find_nash_equilibria(U1, U2):
    m, n = U1.shape  # Number of strategies for Players 1 and 2 respectively
    equilibria = []
    
    for i in range(m):  # Iterate over Player 1's strategies
        for j in range(n):  # Iterate over Player 2's strategies
            # Check if i is a best response for Player 1
            best_response_p1 = U1[i, j] == np.max(U1[:, j])
            # Check if j is a best response for Player 2
            best_response_p2 = U2[i, j] == np.max(U2[i, :])
            
            if best_response_p1 and best_response_p2:
                equilibria.append((i, j))
    
    return equilibria


# Iterate over each combination of conditions and calculate U1
general_tax_scenarios = ["No General Tax", "10% General Tax", "11.25% General Tax"]
low_pricing_allowance_scenarios_SAEV = ["SAEV low pricing allowance 0", "SAEV low pricing allowance 2", "SAEV low pricing allowance 4"]
low_pricing_allowance_scenarios_PT = ["PT low pricing allowance 0", "PT low pricing allowance 1"]
infrastructure_allowance_scenarios = ["No Infrastructure Allowance", "Infrastructure Allowance"]
fixed_allowance_scenarios = ["No Fixed Allowance", "Fixed Allowance"]
variable_allowance_scenarios = ["No Variable Allowance", "33% Variable Allowance", "41% Variable Allowance"]
pricing_scenarios = ["Real Time", "Spatial Temporal"]
rebalancing_options = ["Non-Rebalancing", "Rebalancing"]
discharging_options = ["Non-Discharging", "Discharging"]

results = []

for general_tax in general_tax_scenarios:
    for low_pricing_allowance_SAEV in low_pricing_allowance_scenarios_SAEV:
        for low_pricing_allowance_PT in low_pricing_allowance_scenarios_PT:
            for infrastructure_allowance in infrastructure_allowance_scenarios:
                for fixed_allowance in fixed_allowance_scenarios:
                    for variable_allowance in variable_allowance_scenarios:
                        for pricing_scenario in pricing_scenarios:
                            for rebalancing_option in rebalancing_options:
                                for discharging_option in discharging_options:
                                    if general_tax == "11.25% General Tax" and infrastructure_allowance == "No Infrastructure Allowance" and fixed_allowance == "No Fixed Allowance" and variable_allowance == "No Variable Allowance" and rebalancing_option == "Rebalancing" and discharging_option == "Discharging":
                                        
                                        print(f"{general_tax} - {low_pricing_allowance_SAEV} - {low_pricing_allowance_PT} - {infrastructure_allowance} - {fixed_allowance} - {variable_allowance} - {pricing_scenario} - {rebalancing_option} - {discharging_option}")
                                        
                                        # Construct U1 matrix for the current combination
                                        U1_1_1 = SAEV_payoff["SAEV Pricing 1.75"]["PT Pricing 2.5"][pricing_scenario][rebalancing_option][discharging_option]["Income (Pre-Tax)"]
                                        U1_1_2 = SAEV_payoff["SAEV Pricing 1.75"]["PT Pricing 3"][pricing_scenario][rebalancing_option][discharging_option]["Income (Pre-Tax)"]
                                        U1_2_1 = SAEV_payoff["SAEV Pricing 2"]["PT Pricing 2.5"][pricing_scenario][rebalancing_option][discharging_option]["Income (Pre-Tax)"]
                                        U1_2_2 = SAEV_payoff["SAEV Pricing 2"]["PT Pricing 3"][pricing_scenario][rebalancing_option][discharging_option]["Income (Pre-Tax)"]

                                        U1 = np.array([[U1_1_1, U1_1_2], [U1_2_1, U1_2_2]])
                                        U2 = np.copy(U2_full)
                                        allowance_paid_1 = 0
                                        allowance_paid_2 = 0
                                        tax_U1_high_pricing_SAEV = 0
                                        tax_U2_high_pricing_PT = 0

                                        if infrastructure_allowance == "Infrastructure Allowance":
                                            U2 += infrastructure_cost_matrix

                                        if fixed_allowance == "Fixed Allowance":
                                            U2 += fix_allowance_amount
                                            
                                        if variable_allowance == "33% Variable Allowance":
                                            U2 += 0.33 * variable_cost_matrix
                                        elif variable_allowance == "41% Variable Allowance":
                                            U2 += 0.41 * variable_cost_matrix

                                        if low_pricing_allowance_SAEV == "SAEV low pricing allowance 2":
                                            U1 += np.array([[2,2],[0,0]])
                                            allowance_paid_1 += 2
                                        elif low_pricing_allowance_SAEV == "SAEV low pricing allowance 4":
                                            U1 += np.array([[4,4],[0,0]])
                                            allowance_paid_1 += 4

                                        if low_pricing_allowance_PT == "PT low pricing allowance 1":
                                            U2 += np.array([[1,0],[1,0]])
                                            allowance_paid_2 += 1

                                        # Apply tax policy adjustments
                                        if general_tax == "10% General Tax":
                                            U1, tax_U1_general_matrix = apply_general_tax(U1, 0.1)
                                            U2, tax_U2_general_matrix = apply_general_tax(U2, 0.1)
                                        elif general_tax == "11.25% General Tax":
                                            U1, tax_U1_general_matrix = apply_general_tax(U1, 0.1125)
                                            U2, tax_U2_general_matrix = apply_general_tax(U2, 0.1125)
                                        else:
                                            tax_U1_general_matrix = np.zeros_like(U1)
                                            tax_U2_general_matrix = np.zeros_like(U2)                                

                                        equilibria = find_nash_equilibria(U1, U2)

                                        print(U1)
                                        print(U2)

                                        for eq in equilibria:
                                            i, j = eq
                                            if infrastructure_allowance == "Infrastructure Allowance":
                                                allowance_paid_2 += infrastructure_cost_matrix[i,j]
                                            if fixed_allowance == "Fixed Allowance":
                                                allowance_paid_2 += fix_allowance_amount
                                            if variable_allowance == "33% Variable Allowance":
                                                allowance_paid_2 += 0.33 * variable_cost_matrix[i,j]
                                            elif variable_allowance == "41% Variable Allowance":
                                                allowance_paid_2 += 0.41 * variable_cost_matrix[i,j]

                                            # Calculate the tax at the Nash equilibrium point
                                            tax_U1_general = tax_U1_general_matrix[i, j]
                                            tax_U2_general = tax_U2_general_matrix[i, j]

                                            # Calculate government economic payoff
                                            gov_economic_payoff = (tax_U1_general + tax_U2_general + tax_U1_high_pricing_SAEV + tax_U2_high_pricing_PT - allowance_paid_1 - allowance_paid_2)
                                            gov_environment_payoff = gov_env_matrix[i,j]
                                            gov_payoff = gov_economic_payoff + gov_environment_payoff

                                            results.append({
                                                "SAEV Low Pricing Allowance": int(low_pricing_allowance_SAEV.split()[-1]),
                                                "PT Low Pricing Allowance": float(low_pricing_allowance_PT.split()[-1]),
                                                "Pricing Scenario": pricing_scenario,
                                                "Nash Equilibrium": eq,
                                                "General Tax levied on Player 1": tax_U1_general,
                                                "General Tax levied on Player 2": tax_U2_general,
                                                "Allowance Paid to Player 1": allowance_paid_1,
                                                "Allowance Paid to Player 2": allowance_paid_2,
                                                "Government Economic Payoff": gov_economic_payoff,
                                                "Government Environment Payoff": gov_environment_payoff,
                                                "Government Payoff": gov_payoff
                                            })

# Convert results to a DataFrame and display it
results_df = pd.DataFrame(results)


11.25% General Tax - SAEV low pricing allowance 0 - PT low pricing allowance 0 - No Infrastructure Allowance - No Fixed Allowance - No Variable Allowance - Real Time - Rebalancing - Discharging
[[23.811625 23.998   ]
 [25.32925  26.864625]]
[[-1.69633131 -1.60622655]
 [-1.91569228 -1.95686181]]
11.25% General Tax - SAEV low pricing allowance 0 - PT low pricing allowance 0 - No Infrastructure Allowance - No Fixed Allowance - No Variable Allowance - Spatial Temporal - Rebalancing - Discharging
[[25.355875 25.533375]
 [26.704875 28.337875]]
[[-1.69633131 -1.60622655]
 [-1.91569228 -1.95686181]]
11.25% General Tax - SAEV low pricing allowance 0 - PT low pricing allowance 1 - No Infrastructure Allowance - No Fixed Allowance - No Variable Allowance - Real Time - Rebalancing - Discharging
[[23.811625 23.998   ]
 [25.32925  26.864625]]
[[-0.69633131 -1.60622655]
 [-0.91569228 -1.95686181]]
11.25% General Tax - SAEV low pricing allowance 0 - PT low pricing allowance 1 - No Infrastructure Allowa

In [4]:
results_df

,SAEV Low Pricing Allowance,PT Low Pricing Allowance,Pricing Scenario,Nash Equilibrium,General Tax levied on Player 1,General Tax levied on Player 2,Allowance Paid to Player 1,Allowance Paid to Player 2,Government Economic Payoff,Government Environment Payoff,Government Payoff
0,0,0.0,Real Time,"(1, 0)",3.210750,0.0,0,0,3.210750,-0.022935,3.187815
1,0,0.0,Spatial Temporal,"(1, 0)",3.385125,0.0,0,0,3.385125,-0.022935,3.362190
2,0,1.0,Real Time,"(1, 0)",3.210750,0.0,0,1,2.210750,-0.022935,2.187815
3,0,1.0,Spatial Temporal,"(1, 0)",3.385125,0.0,0,1,2.385125,-0.022935,2.362190
4,2,1.0,Real Time,"(0, 0)",3.243375,0.0,2,1,0.243375,-0.024068,0.219307
5,2,1.0,Spatial Temporal,"(0, 0)",3.439125,0.0,2,1,0.439125,-0.024068,0.415057
6,4,0.0,Real Time,"(0, 1)",3.492000,0.0,4,0,-0.508000,-0.024156,-0.532156
7,4,0.0,Spatial Temporal,"(0, 1)",3.686625,0.0,4,0,-0.313375,-0.024156,-0.337531
8,4,1.0,Real Time,"(0, 0)",3.468375,0.0,4,1,-1.531625,-0.024068,-1.555693
9,4,1.0,Spatial Temporal,"(0, 0)",3.664125,0.0,4,1,-1.335875,-0.024068,-1.359943


### Final Remark: 

In this version, Government payoffs only considers economic payoffs and environment payoffs, thus leading to the result of minimizing allowances. Here are two ideas to make the results more resonable: 1. add social wellfare/resident satisfication into government payoffs, such as low pricing strategies lead to high social wellfare/resident satisfication payoffs; 2. consider the social economic benefit after paying the allowances, an example is: London government paid TfL more than 24m GBP for off-peak underground pricing strategies during Friday peak hours, to encourage citizens going out on Friday afternoons, aims at improving social economic benefits. But a probelm here is that travelling demands are assumed to be fixed during the calculations here, which can make this statement contradicts to itself.